In [1]:
import pandas as pd
import requests

In [2]:
df=pd.read_excel('база предприятий спб/Санкт-Петербург 2ГИС.xlsx')  # Это то, куда вы скачали файл
df.info()


In [3]:
# Удаление строк в которых отсутствует адрес сайта
df.dropna(subset=['Сайт'], inplace=True)

In [4]:
# Удаление лишних столбцов
df.drop(df.columns[range(8,46)], axis=1, inplace=True)

In [3]:
df.info()
df.head()

In [6]:
df.describe(include='all')

In [7]:
# уникальных адресов сайтов 40317 шт, удалит дубликаты
df=df.drop_duplicates('Сайт')
df.info()
df.head()

In [15]:
Сбросить индекс строк
df=df.reset_index(drop=True)
делать не обязательно, так как новый скрипт сам определяет индексы.

In [5]:
df.head(10)

In [7]:
# Парсинг актуального адреса сайта с сохранением истории редиректов.
# Данный способ лучше, так как не нужно каждый раз определять место на котором прервался и с которого нужно начинать.

# Второй и последующие запуски

df=pd.read_pickle('https.pkl')

from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
import threading


#определяем индексы пустых ячеек
index = df[df['url'].isna()].index

#количество потоков
n=10

#создание функции 
def urlverfctn(i):
    """
    i - номер строки в датафреме
    """
    try:
        url=df.loc[i,'Сайт']
        r = requests.head(url, timeout=(30, 200), allow_redirects=True)
        df.loc[i,'url']= r.url
        history=''
        if len(r.history)>0:
            for h in range(len(r.history)):
                history=history + r.history[h].url+"-"+str(r.history[h].status_code)+'->'
            history=history+r.url
        df.loc[i,'history']= history
    except Exception as e:
        df.loc[i,'url']= ("Error! " + str(e))
        print("Error! " + str(e))
    print('OK', i, history)

    
# Настройка потоков  
for i in tnrange(0, len(index), n):
    #массив для добавления потоков
    threads = []
    # переменная для задания диапазона строк
    m=i+n
    for j in range(i,m):
        # Подготавливаем потоки, складываем их в массив
        threads.append(threading.Thread(target=urlverfctn, args=(index[j],)))
    # Запускаем каждый поток
    for thread in threads:
        thread.start()
    # Завершение каждого потока
    for thread in threads:
        thread.join()
    df.to_pickle('https.pkl')

In [8]:
# Проверка адресов, которые выдали ошибку.
# Парсинг актуального адреса сайта с сохранением истории редиректов.

df=pd.read_pickle('https.pkl')

from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
import threading
import logging
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(format = u'%(levelname)-8s [%(asctime)s] %(message)s', level = logging.I, filename = u'mylog.log')


#определяем индексы ячеек с ошибками 
index = df[df['url'].str.find('Error!')>-1][5100:].index

#количество потоков
n=10

#создание функции 
def urlverfctn(i):
    """
    i - номер строки в датафреме
    """
    try:
        url=df.loc[i,'Сайт']
        r = requests.head(url, timeout=(30, 200), allow_redirects=True)
        df.loc[i,'url']= r.url
        history=''
        if len(r.history)>0:
            for h in range(len(r.history)):
                history=history + r.history[h].url+"-"+str(r.history[h].status_code)+'->'
            history=history+r.url
        df.loc[i,'history']= history
    except Exception as e:
        df.loc[i,'url']= ("Error! " + str(e))
        logging.error("Error! " + str(e), i)
    logging.info('OK', i, history)

    
# Настройка потоков  
for i in tnrange(0, len(index), n):
    #массив для добавления потоков
    threads = []
    # переменная для задания диапазона строк
    m=i+n
    for j in range(i,m):
        # Подготавливаем потоки, складываем их в массив
        threads.append(threading.Thread(target=urlverfctn, args=(index[j],)))
    # Запускаем каждый поток
    for thread in threads:
        thread.start()
    # Завершение каждого потока
    for thread in threads:
        thread.join()
    df.to_pickle('https.pkl')

In [2]:
df=pd.read_pickle('https_clean.pkl')

In [12]:
# df1=df[df['url'].str.find('Error!')>-1].reset_index()

In [10]:
# df1[df1['url'].str.find('www.bsc-ltd.ru')>-1]

In [11]:
# df[df['url'].str.find('Error!')>-1][5100:]


In [20]:
# df[df['url'].str.find('Error!')>-1]

In [21]:
# df[df['url'].str.find('Error!')>-1].index

In [26]:
# df.to_pickle('https.pkl')

In [ ]:
# df['https']=df['Сайт'].str.find('https')>-1

In [12]:
df['https']=df['url'].str.find('https')>-1

In [13]:
df['https'].value_counts()


False    32672
True      7645
Name: https, dtype: int64

In [17]:
df=df.drop(df[df['Рубрика']=='Город / Власть'].index)
df.head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
0,Санкт-Петербург,"Аварийно-диспетчерская служба Невского района,...","Дыбенко, 25 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.gks1nev.ru,NaN,(812) 308-88-01,https://www.gks1nev.ru/,http://www.gks1nev.ru/-301->https://www.gks1ne...,True
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False


In [4]:
df['Рубрика'].unique()

array(['Аварийные / справочные / экстренные службы',
       'Автосервис / Автотовары',
       'Досуг / Развлечения / Общественное питание',
       'Интернет / Связь / Информационные технологии',
       'Коммунальные / бытовые / ритуальные услуги',
       'Компьютеры / Бытовая техника / Офисная техника',
       'Культура / Искусство / Религия', 'Мебель / Материалы / Фурнитура',
       'Медицина / Здоровье / Красота', 'Металлы / Топливо / Химия',
       'Оборудование / Инструмент', 'Образование / Работа / Карьера',
       'Одежда / Обувь', 'Охрана / Безопасность',
       'Продукты питания / Напитки', 'Реклама / Полиграфия / СМИ',
       'Спорт / Отдых / Туризм', 'Строительные / отделочные материалы',
       'Строительство / Недвижимость / Ремонт',
       'Текстиль / Предметы интерьера',
       'Товары для животных / Ветеринария',
       'Торговые комплексы / Спецмагазины', 'Транспорт / Грузоперевозки',
       'Хозтовары / Канцелярия / Упаковка',
       'Электроника / Электротехника',
   

In [25]:
df[df['Рубрика']=='Аварийные / справочные / экстренные службы'].head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
0,Санкт-Петербург,"Аварийно-диспетчерская служба Невского района,...","Дыбенко, 25 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.gks1nev.ru,NaN,(812) 308-88-01,https://www.gks1nev.ru/,http://www.gks1nev.ru/-301->https://www.gks1ne...,True
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False


In [9]:
df[df['Подрубрика']=='Аварийные службы']

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
0,Санкт-Петербург,"Аварийно-диспетчерская служба Невского района,...","Дыбенко, 25 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.gks1nev.ru,NaN,(812) 308-88-01,https://www.gks1nev.ru/,http://www.gks1nev.ru/-301->https://www.gks1ne...,True
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False


In [12]:
df=df.drop(df[df['Рубрика']=='Аварийные службы'].index)
df[df['Рубрика']=='Аварийные / справочные / экстренные службы']['Подрубрика'].unique()

array(['Аварийные службы', 'Справочно-информационные услуги',
       'Управление ГО и ЧС',
       'Ликвидация нефтеразливов / газонефтеводопроявлений',
       'Скорая медицинская помощь', 'Службы спасения', 'Телефоны доверия',
       'Эвакуация автомобилей', 'Службы экстренного вызова'], dtype=object)

In [14]:
df=df.drop(df[df['Подрубрика']=='Управление ГО и ЧС'].index)
df[df['Подрубрика']=='Управление ГО и ЧС']

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https


In [16]:
df[df['Подрубрика']=='Ликвидация нефтеразливов / газонефтеводопроявлений']
df=df.drop(df[df['Подрубрика']=='Ликвидация нефтеразливов / газонефтеводопроявлений'].index)

In [22]:
df=df.drop(df[df['Подрубрика']=='Телефоны доверия'].index)

In [26]:
df[df['Рубрика']=='Интернет / Связь / Информационные технологии'].head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
5095,Санкт-Петербург,"Дом.ru Бизнес, телекоммуникационный центр, фил...","Коломяжский проспект, 27",Интернет / Связь / Информационные технологии,IP-телефония,http://www.b2b.domru.ru,b2b@ertelecom.ru,8-800-333-90-00,https://interzet.domru.ru/,http://www.b2b.domru.ru/-301->http://domru.ru-...,True
5096,Санкт-Петербург,"ЭлектронТелеком, телекоммуникационная компания","Коломяжский проспект, 18",Интернет / Связь / Информационные технологии,IP-телефония,http://www.etelecom.ru,info@etelecom.ru;et@etelecom.ru,(812) 602-26-26,http://www.etelecom.ru/,,False
5097,Санкт-Петербург,"MNS, ООО Матрикс Интернет Клуб","Комендантский проспект, 28 к2",Интернет / Связь / Информационные технологии,IP-телефония,http://www.mns.ru,sales@mns.ru,(812) 438-46-00,http://www.mns.ru/,,False
5098,Санкт-Петербург,"КОМСТАР-Регионы, телекоммуникационная компания","Просвещения проспект, 85",Интернет / Связь / Информационные технологии,IP-телефония,http://www.dom.mts.ru,oplata@spb.mts.ru,8-800-250-08-90,https://spb.mts.ru:443/personal/dom/,http://www.dom.mts.ru/-302->http://www.mts.ru/...,True
5099,Санкт-Петербург,"Петербургская Интернет Сеть, телекоммуникацион...","Обуховской Обороны проспект, 120 лит Б",Интернет / Связь / Информационные технологии,IP-телефония,http://www.pinspb.ru,info@pinspb.ru,(812) 677-25-25,https://pinspb.ru//,http://www.pinspb.ru/-301->https://pinspb.ru//,True


In [28]:
df[df['Рубрика']=='Коммунальные / бытовые / ритуальные услуги'].head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
7194,Санкт-Петербург,"Мастер Минутка, ЗАО, сеть экспресс-мастерских","Новаторов бульвар, 11 к2 лит А",Коммунальные / бытовые / ритуальные услуги,Ремонт часов,http://www.masterminutka.ru,info@masterminutka.ru,(812) 448-05-50,http://masterminutka.ru/,http://www.masterminutka.ru/-301->http://maste...,False
7195,Санкт-Петербург,"Aurum, сеть ювелирных и часовых мастерских","Каменноостровский проспект, 42",Коммунальные / бытовые / ритуальные услуги,Ремонт часов,http://www.au-spb.ru,NaN,(812) 715-66-05,http://au-spb.ru/,http://www.au-spb.ru/-301->http://au-spb.ru/,False
7196,Санкт-Петербург,"Дом быта, ООО СТС","Луначарского проспект, 15 к1",Коммунальные / бытовые / ритуальные услуги,Ремонт часов,http://dombyta-spb.ru,spb.dombyta@mail.ru,(812) 491-11-47,http://dombyta-spb.ru/,,False
7197,Санкт-Петербург,"Сеть мастерских, ИП Артюхов Н.И.","Ленинский проспект, 129 к6",Коммунальные / бытовые / ритуальные услуги,Ремонт часов,http://www.clockhospital.narod2.ru,nikarti60@mail.ru,(812) 996-89-15,http://clockhospital.narod.ru/,http://www.clockhospital.narod2.ru/-301->http:...,False
7198,Санкт-Петербург,"Сититайм, сеть мастерских-салонов часов","Садовая, 113/111",Коммунальные / бытовые / ритуальные услуги,Ремонт часов,http://www.citytime.ru,siteinfo@citytime.ru,(812) 275-55-55,https://www.citytime.ru/,http://www.citytime.ru/-301->https://www.cityt...,True


In [30]:
rub=df['Рубрика'].unique()
rub

array(['Аварийные / справочные / экстренные службы',
       'Автосервис / Автотовары',
       'Досуг / Развлечения / Общественное питание',
       'Интернет / Связь / Информационные технологии',
       'Коммунальные / бытовые / ритуальные услуги',
       'Компьютеры / Бытовая техника / Офисная техника',
       'Культура / Искусство / Религия', 'Мебель / Материалы / Фурнитура',
       'Медицина / Здоровье / Красота', 'Металлы / Топливо / Химия',
       'Оборудование / Инструмент', 'Образование / Работа / Карьера',
       'Одежда / Обувь', 'Охрана / Безопасность',
       'Продукты питания / Напитки', 'Реклама / Полиграфия / СМИ',
       'Спорт / Отдых / Туризм', 'Строительные / отделочные материалы',
       'Строительство / Недвижимость / Ремонт',
       'Текстиль / Предметы интерьера',
       'Товары для животных / Ветеринария',
       'Торговые комплексы / Спецмагазины', 'Транспорт / Грузоперевозки',
       'Хозтовары / Канцелярия / Упаковка',
       'Электроника / Электротехника',
   

In [41]:
df[df['Рубрика']==rub[8]]

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
11283,Санкт-Петербург,"Леди, сеть салонов красоты","Новоизмайловский проспект, 4",Медицина / Здоровье / Красота,Ногтевые студии,http://salonyledi.ru,NaN,(812) 369-53-01,http://salonyledi.ru/,,False
11284,Санкт-Петербург,"Ив Роше, сеть студий красоты","Средний проспект В.О., 29",Медицина / Здоровье / Красота,Ногтевые студии,http://www.yves-rocher.ru,NaN,(812) 334-77-04,https://www.yves-rocher.ru/,http://www.yves-rocher.ru/-301->https://www.yv...,True
11285,Санкт-Петербург,"World Class, сеть центров красоты и здоровья","Савушкина, 126",Медицина / Здоровье / Красота,Ногтевые студии,http://www.worldclass.ru,NaN,(812) 313-00-00,https://www.worldclass.ru/,http://www.worldclass.ru/-301->https://www.wor...,True
11286,Санкт-Петербург,"Chicago, салон красоты","Яхтенная, 3 к1",Медицина / Здоровье / Красота,Ногтевые студии,http://www.salon-chicago.ru,mail@salon-chicago.ru,(812) 431-20-02,http://salon-chicago.ru/,http://www.salon-chicago.ru/-301->http://salon...,False
11287,Санкт-Петербург,"Bonheur Jeunesse, SPA-салон","7-я линия В.О., 40",Медицина / Здоровье / Красота,Ногтевые студии,http://bonspa.ru,info@bonspa.ru,(812) 958-02-57,http://bonspa.ru/,,False
11288,Санкт-Петербург,"Caresalon, салон-парикмахерская","Бринько пер, 2",Медицина / Здоровье / Красота,Ногтевые студии,http://www.strizhka-gorod.ru,NaN,(812) 310-68-35,https://strizhka-gorod.ru/,http://www.strizhka-gorod.ru/-301->https://www...,True
11289,Санкт-Петербург,"ЛиЛу, салон красоты","Авиаконструкторов проспект, 2",Медицина / Здоровье / Красота,Ногтевые студии,http://salon-lilu.spb.ru,NaN,(812) 432-42-01,http://salon-lilu.spb.ru/,,False
11290,Санкт-Петербург,"Lux, салон красоты","Серпуховская, 11",Медицина / Здоровье / Красота,Ногтевые студии,http://www.margolin-lux.ru,NaN,(812) 942-64-95,Error! HTTPConnectionPool(host='www.margolin-l...,NaN,False
11291,Санкт-Петербург,"Пудра, сеть студий красоты","Авиаконструкторов проспект, 20 к1",Медицина / Здоровье / Красота,Ногтевые студии,http://www.pudra2.ru,info@pudra2.ru,(812) 340-88-45,http://www.pudra2.ru/,,False
11292,Санкт-Петербург,"Russoni, студия красоты","Партизана Германа, 8 к1",Медицина / Здоровье / Красота,Ногтевые студии,http://www.russoni.ru,NaN,(812) 745-92-17,http://www.russoni.ru/,,False


In [43]:
df.describe()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
count,39537,39537,38626,39537,39537,39537,32919,32899,39535,32208,39537
unique,1,39392,13317,26,949,39537,32693,32445,38672,13461,2
top,Санкт-Петербург,"СПбГУ, Санкт-Петербургский государственный уни...","Варшавская, 3 к1",Строительство / Недвижимость / Ремонт,Туристические агентства,http://www.autoralf.ru,cardiomedika@inbox.ru,(812) 328-20-00,"Error! HTTPSConnectionPool(host='faitid.org', ...",,False
freq,39537,15,125,3201,867,1,6,15,411,18748,32036


In [46]:
# Удаление строк в которых отсутствует адрес почты
df.dropna(subset=['Почта'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32919 entries, 1 to 40316
Data columns (total 11 columns):
Город           32919 non-null object
Наименование    32919 non-null object
Адрес           32346 non-null object
Рубрика         32919 non-null object
Подрубрика      32919 non-null object
Сайт            32919 non-null object
Почта           32919 non-null object
Тел. 1          27075 non-null object
url             32917 non-null object
history         27065 non-null object
https           32919 non-null bool
dtypes: bool(1), object(10)
memory usage: 2.8+ MB


In [51]:
df['https'].value_counts()

False    26670
True      6023
Name: https, dtype: int64

In [52]:
# уникальных адресов сайтов 40317 шт, удалит дубликатов почты
df=df.drop_duplicates('Почта')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32693 entries, 1 to 40316
Data columns (total 11 columns):
Город           32693 non-null object
Наименование    32693 non-null object
Адрес           32125 non-null object
Рубрика         32693 non-null object
Подрубрика      32693 non-null object
Сайт            32693 non-null object
Почта           32693 non-null object
Тел. 1          26876 non-null object
url             32691 non-null object
history         26885 non-null object
https           32693 non-null bool
dtypes: bool(1), object(10)
memory usage: 2.8+ MB


In [118]:
df.to_pickle('https_clean.pkl')

In [119]:
df[df['https']==False]

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False
9,Санкт-Петербург,"ГТИБ, Городское туристско-информационное бюро","Садовая, 37",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.ispb.info,info@ispb.info,(812) 310-22-31,http://ispb.info/,http://www.ispb.info/-301->http://ispb.info/,False
10,Санкт-Петербург,"Экми, справочная о наличии лекарств в аптеках",NaN,Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.3809066.spb.ru,3809066@acmespb.ru,(812) 380-90-66,"Error! HTTPSConnectionPool(host='faitid.org', ...",NaN,False
13,Санкт-Петербург,"РСофт, Лтд, ООО, информационно-маркетинговая к...","Лермонтовский проспект, 7а",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.rsoft.ru,advert@rsoft.ru,(812) 714-31-81,http://www.rsoft.ru/,,False
14,Санкт-Петербург,"064, телефонная справочная служба","Профессора Попова, 23",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.064.ru,info@064.ru,0-64,http://www.064.ru/,,False
16,Санкт-Петербург,"Квартирный Вопрос, городская справочная по нед...","Юрия Гагарина проспект, 1",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.tsn.spb.ru,info@tsn.spb.ru,(812) 32-701-32,http://tsn.spb.ru/,http://www.tsn.spb.ru/-301->http://tsn.spb.ru/,False
18,Санкт-Петербург,"050, телефонная справочная служба",NaN,Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.050spb.ru,info@050spb.ru,0-50,http://050spb.ru/,http://www.050spb.ru/-301->http://050spb.ru/,False
19,Санкт-Петербург,"Ресторанный рейтинг, справочная служба по рест...",NaN,Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.3256500.ru,info@3256500.ru,(812) 325-65-00,Error! HTTPConnectionPool(host='www.3256500.ru...,NaN,False
22,Санкт-Петербург,"Северо-Западное ИнфоМедиа, информационно-серви...","Александра Невского площадь, 2",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.infomedia.ru,info@infomedia.ru,(812) 333-03-33,http://www.infomedia.ru/,,False


In [3]:
df=pd.read_pickle('https_clean.pkl')

In [10]:
df[df['https']==False][:500].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 3 to 844
Data columns (total 11 columns):
Город           500 non-null object
Наименование    500 non-null object
Адрес           495 non-null object
Рубрика         500 non-null object
Подрубрика      500 non-null object
Сайт            500 non-null object
Почта           500 non-null object
Тел. 1          460 non-null object
url             500 non-null object
history         374 non-null object
https           500 non-null bool
dtypes: bool(1), object(10)
memory usage: 43.5+ KB


In [11]:
df[df['https']==False][:500].to_csv('https_clean_500.csv')

In [13]:
df['list_ids/id']='mass_mailing.mass_mail_list_1'
df[df['https']==False][:500].to_csv('https_clean_500.csv')


In [8]:
df['list_ids/id']='__export__.mail_mass_mailing_list_4_0f342078'
# df[df['https']==False][500:1500].to_csv('https_clean_2.csv')


In [6]:
df[df['https']==False][500:1500].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 845 to 3193
Data columns (total 13 columns):
Город           1000 non-null object
Наименование    1000 non-null object
Адрес           973 non-null object
Рубрика         1000 non-null object
Подрубрика      1000 non-null object
Сайт            1000 non-null object
Почта           1000 non-null object
Тел. 1          871 non-null object
url             1000 non-null object
history         769 non-null object
https           1000 non-null bool
list_ids/id     1000 non-null object
Error           1000 non-null bool
dtypes: bool(2), object(11)
memory usage: 95.7+ KB


In [15]:
# df['url2']=df['url'].apply(lambda x: x[5:])
# df.head()
df['url'][5:]

9                                        http://ispb.info/
10       Error! HTTPSConnectionPool(host='faitid.org', ...
12                                     https://www.008.ru/
13                                    http://www.rsoft.ru/
14                                      http://www.064.ru/
15                            https://www.gorzdrav.spb.ru/
16                                      http://tsn.spb.ru/
17                                   https://koalamama.ru/
18                                       http://050spb.ru/
19       Error! HTTPConnectionPool(host='www.3256500.ru...
22                                http://www.infomedia.ru/
26       Error! HTTPConnectionPool(host='www.spravka.or...
28                                     https://normdocs.ru
30                                      https://okonka.ru/
37                                http://www.doctor911.ru/
38                            https://www.sogaz-clinic.ru/
40                                  http://prioritet03.r

In [4]:
df[df['url'].str.find('Error!')>-1][['Сайт','url']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5806 entries, 10 to 40307
Data columns (total 2 columns):
Сайт    5806 non-null object
url     5806 non-null object
dtypes: object(2)
memory usage: 136.1+ KB


In [27]:
df['Error']=df['url'].str.find('Error!')>-1

In [5]:
df.head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https,list_ids/id,Error
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True,mass_mailing.mass_mail_list_1,False
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True,mass_mailing.mass_mail_list_1,False
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False,mass_mailing.mass_mail_list_1,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False,mass_mailing.mass_mail_list_1,False
5,Санкт-Петербург,"Пулково, аэропорт","Пулковское шоссе, 41 лит А",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.pulkovoairport.ru,office@pulkovo-airport.com,(812) 704-38-22,https://pulkovoairport.ru/,http://www.pulkovoairport.ru/-301->https://pul...,True,mass_mailing.mass_mail_list_1,False


In [31]:
df[(df['https']==False)&(df['Error']==False)][:500].to_csv('https_clean_500.csv')
df[(df['https']==False)&(df['Error']==False)][:500]

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https,list_ids/id,Error
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False,mass_mailing.mass_mail_list_1,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False,mass_mailing.mass_mail_list_1,False
9,Санкт-Петербург,"ГТИБ, Городское туристско-информационное бюро","Садовая, 37",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.ispb.info,info@ispb.info,(812) 310-22-31,http://ispb.info/,http://www.ispb.info/-301->http://ispb.info/,False,mass_mailing.mass_mail_list_1,False
13,Санкт-Петербург,"РСофт, Лтд, ООО, информационно-маркетинговая к...","Лермонтовский проспект, 7а",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.rsoft.ru,advert@rsoft.ru,(812) 714-31-81,http://www.rsoft.ru/,,False,mass_mailing.mass_mail_list_1,False
14,Санкт-Петербург,"064, телефонная справочная служба","Профессора Попова, 23",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.064.ru,info@064.ru,0-64,http://www.064.ru/,,False,mass_mailing.mass_mail_list_1,False
16,Санкт-Петербург,"Квартирный Вопрос, городская справочная по нед...","Юрия Гагарина проспект, 1",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.tsn.spb.ru,info@tsn.spb.ru,(812) 32-701-32,http://tsn.spb.ru/,http://www.tsn.spb.ru/-301->http://tsn.spb.ru/,False,mass_mailing.mass_mail_list_1,False
18,Санкт-Петербург,"050, телефонная справочная служба",NaN,Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.050spb.ru,info@050spb.ru,0-50,http://050spb.ru/,http://www.050spb.ru/-301->http://050spb.ru/,False,mass_mailing.mass_mail_list_1,False
22,Санкт-Петербург,"Северо-Западное ИнфоМедиа, информационно-серви...","Александра Невского площадь, 2",Аварийные / справочные / экстренные службы,Справочно-информационные услуги,http://www.infomedia.ru,info@infomedia.ru,(812) 333-03-33,http://www.infomedia.ru/,,False,mass_mailing.mass_mail_list_1,False
37,Санкт-Петербург,"Петербургская Неотложка, ООО, частная скорая м...","Маяковского, 21",Аварийные / справочные / экстренные службы,Скорая медицинская помощь,http://www.doctor911.ru,er@doctor911.ru,8-800-333-19-11,http://www.doctor911.ru/,,False,mass_mailing.mass_mail_list_1,False
40,Санкт-Петербург,"Приоритет, частная скорая медицинская помощь","Кондратьевский проспект, 64 к4",Аварийные / справочные / экстренные службы,Скорая медицинская помощь,http://www.prioritet03.ru,help@prioritet03.ru,(812) 313-21-21,http://prioritet03.ru/,http://www.prioritet03.ru/-301->http://priorit...,False,mass_mailing.mass_mail_list_1,False


In [9]:
df[(df['https']==False)&(df['Error']==False)][500:1500].to_csv('https_clean_2.csv')
df[(df['https']==False)&(df['Error']==False)][500:1500]

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https,list_ids/id,Error
1101,Санкт-Петербург,"Авто XXI, автоцентр","Непокорённых проспект, 17 лит В",Автосервис / Автотовары,Автомойки,http://www.auto21.ru,contact@auto21.ru,(812) 972-66-09,http://www.auto21.ru/,,False,__export__.mail_mass_mailing_list_4_0f342078,False
1103,Санкт-Петербург,"Masterparts, автокомплекс","Ветеранов проспект, 140",Автосервис / Автотовары,Автомойки,http://www.shinka07.ru,shina07@list.ru,(812) 736-87-38,http://shinka07.ru/,http://www.shinka07.ru/-301->http://shinka07.ru/,False,__export__.mail_mass_mailing_list_4_0f342078,False
1104,Санкт-Петербург,"Moikoff, автомойка","Добролюбова проспект, 18 лит А",Автосервис / Автотовары,Автомойки,http://www.moikoff.ru,info@moikoff.ru,(812) 997-66-77,http://moikoff.ru/,http://www.moikoff.ru/-301->http://moikoff.ru/,False,__export__.mail_mass_mailing_list_4_0f342078,False
1106,Санкт-Петербург,"L-Car, автоцентр","Октябрьская Набережная, 104 к43 лит А",Автосервис / Автотовары,Автомойки,http://www.l-car.ru,mail@l-car.ru,(812) 980-99-97,http://l-car.ru/,http://www.l-car.ru/-301->http://l-car.ru/,False,__export__.mail_mass_mailing_list_4_0f342078,False
1112,Санкт-Петербург,"Барс, группа компаний","Литовская, 16",Автосервис / Автотовары,Автомойки,http://www.tkbars.com,tk-bars@mail.ru,(812) 322-64-97,http://www.e-gloryon.com,http://www.tkbars.com/-302->http://www.e-glory...,False,__export__.mail_mass_mailing_list_4_0f342078,False
1113,Санкт-Петербург,"Виват-Авто, автоцентр","Уральская, 16Б",Автосервис / Автотовары,Автомойки,http://www.vivat-auto.com,vivat-avto@mail.ru,(812) 319-38-40,http://www.vivat-auto.com/,,False,__export__.mail_mass_mailing_list_4_0f342078,False
1114,Санкт-Петербург,"Angel Auto, автомойка","Чапаева, 9",Автосервис / Автотовары,Автомойки,http://www.aluxury.ru,angel-auto@bk.ru,(812) 454-75-44,http://www.aluxury.ru/,,False,__export__.mail_mass_mailing_list_4_0f342078,False
1115,Санкт-Петербург,"АвтоБлеск 24, выездная химчистка автомобилей","Невский проспект, 5",Автосервис / Автотовары,Автомойки,http://autoblesk24.ru,info@autoblesk24.ru,(812) 925-77-71,http://autoblesk24.ru/,,False,__export__.mail_mass_mailing_list_4_0f342078,False
1122,Санкт-Петербург,"Кармин, ООО, оптово-розничная компания, предст...","Автомобильная, 8",Автосервис / Автотовары,Запчасти для общественного транспорта,http://karminavto.ru,sale@karminavto.ru,8-800-700-49-90,http://www.karminavto.ru/,http://karminavto.ru/-301->http://www.karminav...,False,__export__.mail_mass_mailing_list_4_0f342078,False
1125,Санкт-Петербург,"ВИП авто, торговая компания","Салова, 27 лит Н",Автосервис / Автотовары,Запчасти для общественного транспорта,http://www.nord-yada.ru,info@nord-yada.ru,(812) 493-50-85,http://www.nord-yada.ru/,,False,__export__.mail_mass_mailing_list_4_0f342078,False


In [32]:
df.to_pickle('https_clean.pkl')

In [6]:
df[(df['https']==False)&(df['Error']==False)][1500:].to_csv('https_clean_3.csv')
df[(df['https']==False)&(df['Error']==False)][1500:]

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https,list_ids/id,Error
3949,Санкт-Петербург,"Ком Тур, туристическая компания","Чайковского, 24",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.com-tour.com,info@com-tour.com,(812) 449-08-94,http://www.com-tour.com/,,False,mass_mailing.mass_mail_list_1,False
3951,Санкт-Петербург,"Вояж Ксения, экскурсионное агентство","16-я линия В.О., 21",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.voyage-ksenia.com,hotel979@yandex.ru,(812) 915-00-50,http://www.voyage-ksenia.com/,,False,mass_mailing.mass_mail_list_1,False
3952,Санкт-Петербург,Совет по туризму и экскурсиям Санкт-Петербурга...,"Итальянская, 3 лит К",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.sovettravel.ru,confirm@ct.spb.ru,(812) 703-13-13,http://sovettravel.ru/,http://www.sovettravel.ru/-301->http://sovettr...,False,mass_mailing.mass_mail_list_1,False
3953,Санкт-Петербург,"Золотой Ключ и Ко, многопрофильная компания","Восстания, 40",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.openkey.spb.ru,office@openkey.spb.ru,(812) 948-13-94,http://www.openkey.spb.ru/,,False,mass_mailing.mass_mail_list_1,False
3956,Санкт-Петербург,"Гатчинское Бюро Путешествий и Экскурсий, ООО","Радищева, 6",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.gbp-gatchina.ru,travel.buro@gatchina.ru,8-981-873-55-40,http://www.gbp-gatchina.ru/,,False,mass_mailing.mass_mail_list_1,False
3957,Санкт-Петербург,"Космос, транспортно-экскурсионная компания","13-я линия В.О., 42",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.cosmos-dmc.ru,cosmos@dmc.spb.ru,(812) 327-72-56,http://cosmos-dmc.ru/,http://www.cosmos-dmc.ru/-301->http://cosmos-d...,False,mass_mailing.mass_mail_list_1,False
3960,Санкт-Петербург,"Scantour, транспортно-туристическая компания","Профессора Попова, 8",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.scantour.ru,info@scantour.ru,(812) 969-66-97,http://scantour.ru/,http://www.scantour.ru/-301->http://scantour.ru/,False,mass_mailing.mass_mail_list_1,False
3962,Санкт-Петербург,"Стиль отдыха, компания по организации экскурси...","Лиговский проспект, 43-45",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.spbturs.ru,spbturs@spbturs.ru,(812) 983-12-85,http://www.spbturs.ru/,,False,mass_mailing.mass_mail_list_1,False
3963,Санкт-Петербург,"GAMMAHOTELS, туристическая компания","Невский проспект, 22",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.gammahotels.ru,gamma-piter@mail.ru,(812) 313-17-29,http://www.gammahotels.ru/,,False,mass_mailing.mass_mail_list_1,False
3964,Санкт-Петербург,"ВИД, туристическая фирма","Торжковская, 3",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.vidspb.ru,vidspb@mail.ru,(812) 716-55-61,http://www.vidspb.ru/,,False,mass_mailing.mass_mail_list_1,False


In [7]:
df[(df['https']==False)&(df['Error']==False)][1500:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19357 entries, 3949 to 40316
Data columns (total 13 columns):
Город           19357 non-null object
Наименование    19357 non-null object
Адрес           19051 non-null object
Рубрика         19357 non-null object
Подрубрика      19357 non-null object
Сайт            19357 non-null object
Почта           19357 non-null object
Тел. 1          15606 non-null object
url             19355 non-null object
history         19355 non-null object
https           19357 non-null bool
list_ids/id     19357 non-null object
Error           19357 non-null bool
dtypes: bool(2), object(11)
memory usage: 1.8+ MB


In [19]:
df['list_ids/id']='__export__.mail_mass_mailing_list_5_da0fd9ab'
df[(df['https']==False)&(df['Error']==False)][1500:].head()


,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https,list_ids/id,Error
3949,Санкт-Петербург,"Ком Тур, туристическая компания","Чайковского, 24",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.com-tour.com,info@com-tour.com,(812) 449-08-94,http://www.com-tour.com/,,False,__export__.mail_mass_mailing_list_5_da0fd9ab,False
3951,Санкт-Петербург,"Вояж Ксения, экскурсионное агентство","16-я линия В.О., 21",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.voyage-ksenia.com,hotel979@yandex.ru,(812) 915-00-50,http://www.voyage-ksenia.com/,,False,__export__.mail_mass_mailing_list_5_da0fd9ab,False
3952,Санкт-Петербург,Совет по туризму и экскурсиям Санкт-Петербурга...,"Итальянская, 3 лит К",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.sovettravel.ru,confirm@ct.spb.ru,(812) 703-13-13,http://sovettravel.ru/,http://www.sovettravel.ru/-301->http://sovettr...,False,__export__.mail_mass_mailing_list_5_da0fd9ab,False
3953,Санкт-Петербург,"Золотой Ключ и Ко, многопрофильная компания","Восстания, 40",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.openkey.spb.ru,office@openkey.spb.ru,(812) 948-13-94,http://www.openkey.spb.ru/,,False,__export__.mail_mass_mailing_list_5_da0fd9ab,False
3956,Санкт-Петербург,"Гатчинское Бюро Путешествий и Экскурсий, ООО","Радищева, 6",Досуг / Развлечения / Общественное питание,Организация экскурсий по городу,http://www.gbp-gatchina.ru,travel.buro@gatchina.ru,8-981-873-55-40,http://www.gbp-gatchina.ru/,,False,__export__.mail_mass_mailing_list_5_da0fd9ab,False


In [32]:
header=('Город', 'name', 'Адрес', 'Рубрика', 'Подрубрика', 'x_site', 'email', 'Тел. 1',
        'url', 'history', 'https', 'list_ids/id', 'Error')
df[(df['https']==False)&(df['Error']==False)][1500:].to_csv('https_clean_3.csv', header=header)